# Bronze to Silver Schema Analysis

**Objective**: Analyze and map SalesLT bronze layer structure to retail silver layer data model

**Architecture**:
- **Source**: RDS_Fabric_Foundry_workspace_Gaiye_Retail_Solution_Test_IDM_LH_bronze (10 SalesLT tables)
- **Target**: RDS_Fabric_Foundry_workspace_Gaiye_Retail_Solution_Test_LH_silver (50+ retail model tables)
- **Execution Context**: Run in silver lakehouse to access both bronze (via shortcut) and silver structures

**Key Challenge**: Transform 10 normalized SalesLT tables into 50+ retail model with main entities: order, customer, brandProduct + ID tables

**Workflow**:
1. **Schema Discovery** - Analyze bronze and silver structures
2. **Mapping Analysis** - Identify transformation requirements
3. **Gap Identification** - Document missing data and transformation needs
4. **Transformation Planning** - Create implementation roadmap

## Step 1: Environment Setup

In [ ]:
# Import required libraries
import pandas as pd
from datetime import datetime
from pyspark.sql.functions import *
from pyspark.sql.types import *

# Configuration
BRONZE_DATABASE = "RDS_Fabric_Foundry_workspace_Gaiye_Retail_Solution_Test_IDM_LH_bronze"
SILVER_TARGET_PATH = "Files/Retail/"
SOURCE_SYSTEM = "SalesLT_to_Retail"
LOAD_TIMESTAMP = datetime.now().isoformat()
LOAD_DATE = datetime.now().strftime("%Y-%m-%d")

# Expected bronze tables (SalesLT)
BRONZE_TABLES = [
    'bronze_address', 'bronze_customer', 'bronze_customeraddress', 
    'bronze_product', 'bronze_productcategory', 'bronze_productdescription', 
    'bronze_productmodel', 'bronze_productmodelproductdescription', 
    'bronze_salesorderdetail', 'bronze_salesorderheader'
]

# Key silver target entities
SILVER_MAIN_ENTITIES = ['order', 'customer', 'brandProduct']

print("🔍 BRONZE TO SILVER SCHEMA ANALYSIS")
print("=" * 60)
print(f"✅ Libraries imported")
print(f"📅 Analysis timestamp: {LOAD_TIMESTAMP}")
print(f"📥 Bronze source: {BRONZE_DATABASE}")
print(f"📤 Silver target path: {SILVER_TARGET_PATH}")
print(f"📊 Bronze tables to analyze: {len(BRONZE_TABLES)}")
print(f"🎯 Main silver entities: {', '.join(SILVER_MAIN_ENTITIES)}")
print(f"✅ Microsoft Fabric PySpark environment ready")

In [ ]:
# Optional debugging information 
# Quick connectivity test to verify access to both layers
print("🔗 CONNECTIVITY TEST")
print("=" * 40)

# Test bronze layer access
try:
    bronze_test = spark.sql(f"SHOW TABLES IN {BRONZE_DATABASE}")
    bronze_count = bronze_test.count()
    print(f"✅ Bronze layer accessible: {bronze_count} tables found")
    print(f"   Database: {BRONZE_DATABASE}")
except Exception as e:
    print(f"❌ Bronze layer access failed: {str(e)[:80]}...")

# Test silver layer access (current context)
try:
    silver_test = spark.sql("SHOW TABLES")
    silver_count = silver_test.count()
    print(f"✅ Silver layer accessible: {silver_count} tables found")
    print(f"   Context: Current lakehouse")
except Exception as e:
    print(f"❌ Silver layer access failed: {str(e)[:80]}...")

print(f"\n🚀 Ready to proceed with full analysis!")
print("=" * 40)

## Step 2: Discover Bronze Layer Structure

In [ ]:
# Analyze bronze layer tables and schemas
print("🔍 ANALYZING BRONZE LAYER STRUCTURE")
print("=" * 60)

# Import Python's built-in sum to avoid PySpark conflict
from builtins import sum as python_sum

bronze_schema_analysis = []

print(f"📥 Source: {BRONZE_DATABASE}")
print(f"📋 Expected bronze tables: {len(BRONZE_TABLES)}")
print()

# Check if we can access bronze database
try:
    # Test access to bronze database
    bronze_tables_df = spark.sql(f"SHOW TABLES IN {BRONZE_DATABASE}").toPandas()
    print(f"✅ Bronze database accessible: {len(bronze_tables_df)} tables found")
    
    # Get table column info
    table_column = None
    for possible_col in ['tableName', 'table_name', 'name']:
        if possible_col in bronze_tables_df.columns:
            table_column = possible_col
            break
    
    if table_column is None:
        table_column = bronze_tables_df.columns[0]
    
    available_bronze_tables = bronze_tables_df[table_column].tolist()
    print(f"📋 Available tables: {', '.join(available_bronze_tables)}")
    
except Exception as e:
    print(f"❌ Cannot access bronze database: {str(e)}")
    print("💡 Ensure you have a shortcut to the bronze lakehouse")
    print("💡 Verify you're running in the silver lakehouse context")
    available_bronze_tables = []

print()

# Analyze each bronze table structure
if available_bronze_tables:
    print("📊 BRONZE TABLE SCHEMA ANALYSIS")
    print("-" * 40)
    
    for table_name in BRONZE_TABLES:
        if table_name in available_bronze_tables:
            print(f"\n🔍 Analyzing: {table_name}")
            
            try:
                # Get table schema and sample data
                df = spark.table(f"{BRONZE_DATABASE}.{table_name}")
                row_count = df.count()
                columns = df.columns
                
                # Separate business vs metadata columns
                business_cols = [col for col in columns if not col.startswith('_')]
                metadata_cols = [col for col in columns if col.startswith('_')]
                
                print(f"   📊 Rows: {row_count:,}")
                print(f"   📈 Business columns: {len(business_cols)}")
                print(f"   🏷️ Metadata columns: {len(metadata_cols)}")
                print(f"   📋 Business schema: {', '.join(business_cols[:10])}{'...' if len(business_cols) > 10 else ''}")
                
                # Store schema info
                bronze_schema_analysis.append({
                    "table": table_name,
                    "row_count": row_count,
                    "total_columns": len(columns),
                    "business_columns": business_cols,
                    "metadata_columns": metadata_cols,
                    "schema": df.schema
                })
                
                # Show detailed schema for key tables
                if table_name in ['bronze_customer', 'bronze_product', 'bronze_salesorderheader']:
                    print(f"   📝 Detailed schema:")
                    for field in df.schema.fields:
                        if not field.name.startswith('_'):  # Only business columns
                            print(f"      • {field.name}: {field.dataType}")
                
            except Exception as e:
                print(f"   ❌ Error analyzing {table_name}: {str(e)[:60]}...")
                bronze_schema_analysis.append({
                    "table": table_name,
                    "error": str(e)[:100],
                    "status": "failed"
                })
        else:
            print(f"⚠️ Table not found: {table_name}")

    print(f"\n📊 BRONZE ANALYSIS SUMMARY")
    print("=" * 40)
    successful_analyses = [t for t in bronze_schema_analysis if "error" not in t]
    failed_analyses = [t for t in bronze_schema_analysis if "error" in t]
    
    print(f"✅ Successfully analyzed: {len(successful_analyses)} tables")
    print(f"❌ Failed analyses: {len(failed_analyses)} tables")
    
    if successful_analyses:
        total_rows = python_sum(t["row_count"] for t in successful_analyses)
        total_business_cols = python_sum(len(t["business_columns"]) for t in successful_analyses)
        print(f"📊 Total rows in bronze: {total_rows:,}")
        print(f"📈 Total business columns: {total_business_cols}")
        
        print(f"\n📋 Bronze layer inventory:")
        for analysis in successful_analyses:
            print(f"  • {analysis['table']}: {analysis['row_count']:,} rows, {len(analysis['business_columns'])} business cols")
else:
    print("⚠️ No bronze tables available for analysis")

## Step 3: Discover Silver Layer Structure

In [ ]:
# Analyze current silver layer structure (target schema)
print("🎯 ANALYZING SILVER LAYER STRUCTURE")
print("=" * 60)

silver_schema_analysis = []

print(f"📤 Target: Current silver lakehouse")
print(f"🎯 Looking for retail model tables")
print()

try:
    # Get all tables in current (silver) lakehouse
    silver_tables_df = spark.sql("SHOW TABLES").toPandas()
    
    if len(silver_tables_df) > 0:
        # Handle column naming
        table_column = None
        for possible_col in ['tableName', 'table_name', 'name']:
            if possible_col in silver_tables_df.columns:
                table_column = possible_col
                break
        
        if table_column is None:
            table_column = silver_tables_df.columns[0]
        
        silver_tables = silver_tables_df[table_column].tolist()
        print(f"✅ Silver lakehouse accessible: {len(silver_tables)} tables found")
        
        # Categorize tables
        main_entity_tables = []
        id_lookup_tables = []
        other_tables = []
        
        for table in silver_tables:
            table_lower = table.lower()
            if any(entity in table_lower for entity in SILVER_MAIN_ENTITIES):
                main_entity_tables.append(table)
            elif 'id' in table_lower or table_lower.endswith('type') or table_lower.endswith('category'):
                id_lookup_tables.append(table)
            else:
                other_tables.append(table)
        
        print(f"🎯 Main entity tables: {len(main_entity_tables)}")
        print(f"🔢 ID/Lookup tables: {len(id_lookup_tables)}")
        print(f"📊 Other tables: {len(other_tables)}")
        
        print(f"\n🎯 MAIN ENTITY TABLES:")
        for table in sorted(main_entity_tables):
            print(f"  • {table}")
        
        print(f"\n🔢 ID/LOOKUP TABLES (sample):")
        for table in sorted(id_lookup_tables)[:10]:  # Show first 10
            print(f"  • {table}")
        if len(id_lookup_tables) > 10:
            print(f"  ... and {len(id_lookup_tables) - 10} more")
        
        print(f"\n📊 OTHER TABLES:")
        for table in sorted(other_tables)[:5]:  # Show first 5
            print(f"  • {table}")
        if len(other_tables) > 5:
            print(f"  ... and {len(other_tables) - 5} more")
        
        # Analyze key main entity tables in detail
        print(f"\n📝 DETAILED ANALYSIS OF KEY TABLES")
        print("-" * 40)
        
        key_tables_to_analyze = [t for t in main_entity_tables if any(entity in t.lower() for entity in SILVER_MAIN_ENTITIES)]
        
        for table_name in key_tables_to_analyze[:3]:  # Analyze first 3 key tables
            print(f"\n🔍 Analyzing: {table_name}")
            
            try:
                df = spark.table(table_name)
                row_count = df.count()
                columns = df.columns
                
                print(f"   📊 Rows: {row_count:,}")
                print(f"   📈 Columns: {len(columns)}")
                print(f"   📋 Schema: {', '.join(columns[:8])}{'...' if len(columns) > 8 else ''}")
                
                # Show detailed schema
                print(f"   📝 Detailed schema:")
                for field in df.schema.fields[:10]:  # Show first 10 fields
                    print(f"      • {field.name}: {field.dataType}")
                if len(df.schema.fields) > 10:
                    print(f"      ... and {len(df.schema.fields) - 10} more columns")
                
                # Store schema info
                silver_schema_analysis.append({
                    "table": table_name,
                    "row_count": row_count,
                    "columns": columns,
                    "schema": df.schema,
                    "category": "main_entity"
                })
                
            except Exception as e:
                print(f"   ❌ Error analyzing {table_name}: {str(e)[:60]}...")
        
        # Store summary info
        silver_summary = {
            "total_tables": len(silver_tables),
            "main_entity_tables": main_entity_tables,
            "id_lookup_tables": id_lookup_tables,
            "other_tables": other_tables
        }
        
    else:
        print("📋 No tables found in current silver lakehouse")
        print("💡 This is expected if silver layer hasn't been created yet")
        silver_summary = {"total_tables": 0}

except Exception as e:
    print(f"❌ Error accessing silver lakehouse: {str(e)}")
    silver_summary = {"error": str(e)}

print(f"\n🎯 SILVER ANALYSIS SUMMARY")
print("=" * 40)
if "error" not in silver_summary:
    print(f"📊 Total silver tables: {silver_summary['total_tables']}")
    if silver_summary['total_tables'] > 0:
        print(f"🎯 Main entities: {len(silver_summary['main_entity_tables'])}")
        print(f"🔢 ID/Lookup tables: {len(silver_summary['id_lookup_tables'])}")
        print(f"📊 Other tables: {len(silver_summary['other_tables'])}")
        print(f"✅ Silver structure analysis complete")
    else:
        print(f"📋 Empty silver lakehouse - ready for initial population")
else:
    print(f"⚠️ Silver analysis failed: {silver_summary['error']}")

## Step 4: Schema Mapping Analysis

In [ ]:
# Analyze mapping requirements between bronze (SalesLT) and silver (Retail) schemas
print("🗺️ SCHEMA MAPPING ANALYSIS")
print("=" * 60)

print("📋 BRONZE TO SILVER TRANSFORMATION REQUIREMENTS")
print()

# UPDATED: Based on actual silver schema analysis (43 empty tables with complex structure)
transformation_analysis = {
    "customer_mapping": {
        "bronze_source": "bronze_customer + bronze_customeraddress + bronze_address",
        "silver_target": "Multiple customer tables (CustomerTelephoneNumber, CustomerName, CustomerAccountLocation, etc.)",
        "complexity": "High",
        "notes": "Map simple SalesLT customer to complex temporal retail customer model with separate name/phone/location tables"
    },
    "product_mapping": {
        "bronze_source": "bronze_product + bronze_productcategory + bronze_productmodel + bronze_productdescription",
        "silver_target": "Complex retail product model (likely 10+ related tables)",
        "complexity": "Very High", 
        "notes": "Transform SalesLT product hierarchy to sophisticated retail brandProduct structure with temporal tracking"
    },
    "order_mapping": {
        "bronze_source": "bronze_salesorderheader + bronze_salesorderdetail",
        "silver_target": "Retail order ecosystem (likely 5+ order-related tables)",
        "complexity": "Very High",
        "notes": "Map SalesLT order/detail to complex retail order model with period tracking and preference management"
    },
    "schema_population": {
        "bronze_source": "10 SalesLT tables with real data (3,000+ total rows)",
        "silver_target": "43 empty retail tables with defined schemas",
        "complexity": "Very High",
        "notes": "Populate empty retail schemas with transformed SalesLT data - major data modeling challenge"
    }
}

print("🗺️ TRANSFORMATION MAP:")
for mapping_name, details in transformation_analysis.items():
    print(f"\n📍 {mapping_name.replace('_', ' ').title()}:")
    print(f"   📥 Source: {details['bronze_source']}")
    print(f"   📤 Target: {details['silver_target']}")
    print(f"   🔧 Complexity: {details['complexity']}")
    print(f"   📝 Notes: {details['notes']}")

# Updated challenges based on actual findings: 43 empty silver tables
challenges = [
    "Schema Population: 43 empty retail tables need data from 10 SalesLT tables",
    "Temporal Modeling: Silver expects PeriodStart/End timestamps not in bronze",
    "ID Generation: Need to create complex retail IDs (CustomerNameId, TelephoneNumberTypeId, etc.)",
    "Data Decomposition: Split simple SalesLT fields into multiple retail entities", 
    "Business Logic: Apply retail-specific transformations (preferences, locations, periods)",
    "Schema Complexity: Map simple normalized data to sophisticated retail model"
]

for i, challenge in enumerate(challenges, 1):
    print(f"   {i}. {challenge}")

# Recommend transformation approach
print(f"\n🚀 RECOMMENDED TRANSFORMATION APPROACH:")
print("=" * 40)

recommendations = [
    "Analyze existing silver table schemas in detail (what fields, relationships)",
    "Start with simplest silver table population (find tables with minimal dependencies)",
    "Create ID generation strategy for retail model IDs",
    "Build temporal data logic (PeriodStart/End, preferences)",
    "Focus on core entities first (Customer → Product → Order)",
    "Develop data quality validation for retail constraints"
]

for i, rec in enumerate(recommendations, 1):
    print(f"   {i}. {rec}")

print(f"\n💡 NEXT STEPS:")
print("1. Deep dive into silver table schemas - understand the 43 empty tables")
print("2. Create Silver_Schema_Deep_Analysis.ipynb to map each table")
print("3. Identify easiest tables to populate first (minimal dependencies)")
print("4. Build Customer data population notebook (focus on core customer tables)")
print("5. Create ID generation and temporal data handling utilities")
print("6. Expand to product and order table population")

print(f"\n✅ Schema analysis complete!")
print(f"📋 Ready for deep schema mapping phase")
print(f"🎯 Critical: Understand the 43 empty silver table structures before transformation")

## Step 5: Generate Implementation Plan

In [ ]:
# Generate detailed implementation plan based on analysis
print("📋 IMPLEMENTATION PLAN GENERATION")
print("=" * 60)

# Create implementation roadmap
implementation_plan = {
    "phase_1_customer": {
        "priority": 1,
        "complexity": "Low-Medium",
        "estimated_effort": "1-2 days",
        "bronze_tables": ["bronze_customer", "bronze_customeraddress", "bronze_address"],
        "silver_targets": ["customer", "customer_address", "address_type"],
        "key_transformations": [
            "Join customer with address data",
            "Standardize address formats", 
            "Create customer demographics",
            "Generate customer ID mappings"
        ],
        "success_criteria": "Customer data accessible in silver with proper relationships"
    },
    "phase_2_product": {
        "priority": 2,
        "complexity": "High",
        "estimated_effort": "3-4 days",
        "bronze_tables": ["bronze_product", "bronze_productcategory", "bronze_productmodel", "bronze_productdescription"],
        "silver_targets": ["brandProduct", "product_category", "product_brand", "product_model"],
        "key_transformations": [
            "Map product hierarchy to brand/product model",
            "Extract brand information from product data",
            "Create product categorization structure",
            "Handle product descriptions and attributes"
        ],
        "success_criteria": "Product catalog available in retail brand/product structure"
    },
    "phase_3_orders": {
        "priority": 3,
        "complexity": "High", 
        "estimated_effort": "2-3 days",
        "bronze_tables": ["bronze_salesorderheader", "bronze_salesorderdetail"],
        "silver_targets": ["order", "order_item", "order_status"],
        "key_transformations": [
            "Normalize order header/detail structure",
            "Calculate order totals and metrics",
            "Map sales data to retail order concepts",
            "Create order status and type lookups"
        ],
        "success_criteria": "Order transactions available in retail order structure"
    },
    "phase_4_lookups": {
        "priority": 4,
        "complexity": "Medium",
        "estimated_effort": "1-2 days", 
        "bronze_tables": ["All bronze tables (extract categorical data)"],
        "silver_targets": ["Various ID/lookup tables"],
        "key_transformations": [
            "Extract unique categorical values",
            "Create standardized ID structures",
            "Build reference data tables",
            "Ensure referential integrity"
        ],
        "success_criteria": "Complete lookup table ecosystem supporting main entities"
    }
}

print("🗓️ IMPLEMENTATION ROADMAP:")
total_effort = 0

for phase_name, phase_details in implementation_plan.items():
    effort_days = phase_details["estimated_effort"].split()[0].split('-')[-1]  # Get max days
    total_effort += int(effort_days)
    
    print(f"\n📍 Phase {phase_details['priority']}: {phase_name.replace('_', ' ').title()}")
    print(f"   🔧 Complexity: {phase_details['complexity']}")
    print(f"   ⏱️ Effort: {phase_details['estimated_effort']}")
    print(f"   📥 Bronze inputs: {', '.join(phase_details['bronze_tables'][:3])}{'...' if len(phase_details['bronze_tables']) > 3 else ''}")
    print(f"   📤 Silver outputs: {', '.join(phase_details['silver_targets'])}")
    print(f"   🎯 Success: {phase_details['success_criteria']}")

print(f"\n📊 OVERALL PROJECT ESTIMATE:")
print(f"   ⏱️ Total estimated effort: 7-{total_effort} days")
print(f"   📋 Total phases: {len(implementation_plan)}")
print(f"   🎯 End goal: Complete bronze → silver transformation pipeline")

# Generate specific notebook recommendations
print(f"\n📓 RECOMMENDED NOTEBOOKS:")
notebooks = [
    "Customer_Bronze_to_Silver_Transform.ipynb - Phase 1 implementation",
    "Product_Bronze_to_Silver_Transform.ipynb - Phase 2 implementation", 
    "Order_Bronze_to_Silver_Transform.ipynb - Phase 3 implementation",
    "Lookup_Tables_Generator.ipynb - Phase 4 implementation",
    "Silver_Layer_Validation.ipynb - End-to-end validation",
    "Bronze_Silver_Pipeline_Orchestrator.ipynb - Fabric Pipeline setup"
]

for notebook in notebooks:
    print(f"   📓 {notebook}")

print(f"\n🚀 READY TO START IMPLEMENTATION!")
print("💡 Recommended first step: Create Customer_Bronze_to_Silver_Transform.ipynb")
print("🎯 Focus: Start simple, build confidence, then tackle complex transformations")

---

## Summary

This analysis notebook provides comprehensive understanding of the bronze → silver transformation requirements:

### ✅ **Key Findings**:
- **Bronze Layer**: 10 SalesLT tables with normalized structure
- **Silver Target**: 50+ retail model tables (customer, order, brandProduct + lookups)
- **Transformation Complexity**: High - significant schema and domain model changes required
- **Implementation Strategy**: Phased approach starting with Customer (simplest) → Product/Order (complex) → Lookups

### 🗺️ **Transformation Map**:
- **Customer**: bronze_customer + address tables → customer entity + lookups
- **Product**: bronze_product + related → brandProduct entity + category structure  
- **Order**: bronze_sales tables → order entity + transaction structure
- **Lookups**: Extract categorical data → 40+ ID/reference tables

### 📋 **Implementation Plan**:
- **Phase 1**: Customer transformation (1-2 days) - ✅ **NOTEBOOK READY**
- **Phase 2**: Product/Brand transformation (3-4 days) - *Next to create*
- **Phase 3**: Order transformation (2-3 days) - *Planned*
- **Phase 4**: Lookup table generation (1-2 days) - *Planned*

### 📓 **Implementation Status**:
- ✅ **Customer_Bronze_to_Silver_Transform.ipynb**: Complete framework with structured approach, transformation logic, schema design, data quality validation
- 🔄 **Product_Bronze_to_Silver_Transform.ipynb**: Next phase to implement
- 📋 **Order_Bronze_to_Silver_Transform.ipynb**: Awaiting customer/product completion
- 🔧 **Lookup_Tables_Generator.ipynb**: Final phase for reference data

### 🚀 **Next Steps**:
1. ✅ **Customer_Bronze_to_Silver_Transform.ipynb Created** - Phase 1 implementation ready
2. **Execute customer transformation** - Run the notebook to create first silver tables (847 customers, 450 addresses)
3. **Validate transformation results** - Built-in data quality validation framework
4. **Create Product_Bronze_to_Silver_Transform.ipynb** - Phase 2 implementation
5. **Expand to order transformations and lookup table generation**
6. **Set up Fabric Pipeline orchestration for end-to-end automation**

### 💡 **Success Strategy**:
**Phase 1 Ready**: Customer transformation notebook provides comprehensive framework with ID generation (CUST_xxxxxxxx, ADDR_xxxxxxxx), data enhancement (Individual/Business classification), address standardization, relationship management, and audit trails. Start with customer data to build confidence and establish proven transformation patterns for complex product and order phases.

## Customer Transformation Features

The **Customer_Bronze_to_Silver_Transform.ipynb** notebook provides a complete framework for Phase 1 transformation:

**Core Components:**
- Environment setup and connectivity testing
- Comprehensive bronze customer/address data analysis
- Retail-focused silver schema design with audit trails
- Advanced transformation logic with ID generation
- Quality validation and error handling

**Key Capabilities:**
- Generates standardized retail IDs: CUST_xxxxxxxx and ADDR_xxxxxxxx
- Classifies customers as Individual or Business types
- Standardizes addresses with proper country codes
- Manages primary address relationships and mappings
- Maintains complete audit trails from source systems

**Quality Assurance:**
- ✅ Record count validation
- ✅ Data completeness checks  
- ✅ Referential integrity validation
- ✅ Business rule enforcement

## Execute Phase 1 Transformation

Your bronze layer contains **847 customers**, **450 addresses**, and **450 relationships** ready for transformation.

**Implementation Steps:**
1. **Run Customer Transformation** - Execute the notebook to populate your first silver tables
2. **Validate Results** - Built-in quality checks ensure successful transformation
3. **Build Foundation** - Establish patterns for subsequent Product and Order phases

**Expected Results:**
- ✅ Populated silver customer tables with transformed data
- ✅ Established retail ID structure (CUST_ and ADDR_ prefixes)  
- ✅ Standardized addresses and relationship mappings
- ✅ Proven framework ready for Phase 2 (Products) and Phase 3 (Orders)

The customer transformation creates the foundation patterns that will be used for all subsequent bronze-to-silver transformations in your data pipeline.